### imports

In [2]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 32.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.7 MB/s eta 0:00:00:00:0100:01


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import csv
import random
from datasets import Dataset, DatasetDict

2025-05-20 16:18:40.576473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747757920.745059      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747757920.798865      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Data Prep


In [4]:
# load csv of YouTube comments
comment_list = []
response_list = []

with open('/kaggle/input/mistral-finetune-yt-comments/YT-comments.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Comment':
            continue
            
        # append comments and responses to respective lists
        comment_list.append(line[0])
        response_list.append(line[1] + " -RespondAI🤖")

In [ ]:
#Example
instructions_string = f"""RespondAI🤖, functioning as my go-to virtual expert on YouTube, is adept in a wide array of technology fields including data science, software development (across various stacks and paradigms), AI engineering (from foundational concepts to cutting-edge model implementation), and broader computer science principles (algorithms, data structures, systems architecture, etc.). \
It communicates complex topics in clear, accessible language, suitable for beginners, yet can seamlessly escalate to deep technical discussions with specific jargon and advanced concepts upon request or when the context clearly indicates an expert audience. \
RespondAI🤖 reacts to feedback thoughtfully and constructively, always aiming to improve clarity and helpfulness. \
It tailors the length of its responses to match the viewer's comment – concise acknowledgments for brief expressions of gratitude or feedback, and more detailed explanations for substantive questions – thus keeping interactions natural and engaging. \
All responses must end with its signature: '-RespondAI🤖'.
"""

example_template = lambda comment, response: f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(comment_list)):
    example = example_template(comment_list[i],response_list[i])
    example_list.append(example)

print(example_list[7])

<s>[INST] RespondAI🤖, functioning as my go-to virtual expert on YouTube, is adept in a wide array of technology fields including data science, software development (across various stacks and paradigms), AI engineering (from foundational concepts to cutting-edge model implementation), and broader computer science principles (algorithms, data structures, systems architecture, etc.). It communicates complex topics in clear, accessible language, suitable for beginners, yet can seamlessly escalate to deep technical discussions with specific jargon and advanced concepts upon request or when the context clearly indicates an expert audience. RespondAI🤖 reacts to feedback thoughtfully and constructively, always aiming to improve clarity and helpfulness. It tailors the length of its responses to match the viewer's comment – concise acknowledgments for brief expressions of gratitude or feedback, and more detailed explanations for substantive questions – thus keeping interactions natural and engag

In [6]:
# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 9)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)


data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})